In [1]:
%config IPCompleter.greedy=True
import re
import json
from sklearn.feature_extraction.text import CountVectorizer
from functools import partial

Read dataset

In [2]:
cv = CountVectorizer(token_pattern=r'\w+')
analyzer = cv.build_analyzer()

In [3]:
full_q = []
full_q_freq = []
with open('yaqq_full.tsv', 'r') as inf:
    for line in inf:
        q = ' '.join(line.strip().split()[:-1])
        full_q.append(' '.join(analyzer(q)))
        full_q_freq.append(int(line.strip().split()[-1]))

### Filtering

Filters

In [139]:
# all filters

def filter_shorter_than_n(q, n=2):
    q = ' '.join(analyzer(q))
    return len(q.split()) > n

def filter_starts_with_how(q):
    q = ' '.join(analyzer(q))
    words = q.split()
    if words[0] != 'как':
        return True
    whitelist = [
        'зовут',
        'звали',
        'называется',
        'называли',
        'называют',
        'называлось',
        'назывались',
        'называлась',
        'назывался',
        'называются'
    ]
    if words[1] in whitelist:
        return True
    return False

def filter_starts_with_what(q):
    q = ' '.join(analyzer(q))
    words = q.split()
    if words[0] != 'что':
        return True
    blacklist = [
        'будет',
        'нужно',
        'делать',
        'можно',
        'лучше'
    ]
    if words[1] in blacklist:
        return False
    return True

def filter_stop_words(q):
    q = ' '.join(analyzer(q))
    stop_words = set([
        'купить',
        'почему',
        'онлайн',
        'сегодня',
        'лучше',
        'зачем',
        'снится',
        'приготовить',
        'беременности',
        'видео',
        'документы',
        'тест', 
        'подарить', 
        'снятся', 
        'калорий',
        'беременным', 
        'бесплатно', 
        'нужен',
        'нужна',
        'нужно',
        'нужны',
        'отдохнуть', 
        'сниться', 
        'нельзя', 
        'заниматься', 
        'беременность', 
        'отдыхать', 
        'выбрать',
        'забеременеть', 
        'месячных', 
        'сексом', 
        'похудеть', 
        'контакте', 
        'загранпаспорт',
        'сайт',
        'может',
        'фото'
    ])
    words = q.split()
    for word in words:
        if word in stop_words:
            return False
    return True

def filter_stop_substr(q):
    q = ' '.join(analyzer(q))
    stop_substr = [
        '100 к 1',
        'сто к одному',
        'можно ли',
        'где можно'
    ]
    for sb in stop_substr:
        if sb in q:
            return False
    return True

def filter_starts_with_stop_words(q):
    q = ' '.join(analyzer(q))
    stop_words = [
        'игра',
        'игры',
        'с чем',
        'чем ',
        'сколько стоит',
        'где отметить',
        'где отпраздновать',
        'что подарить'
    ]
    for sw in stop_words:
        if q.startswith(sw):
            return False
    return True

def filter_must_start_with_question_word(q):
    ws = ' '.join(analyzer(q))
    q_words = [
        'кто ',
        'где ',
        'когда ',
        'какой ',
        'каком '
    ]
    for qw in q_words:
        if qw in ws:
            return True
    return False
    

def filter_unknown_words(q):
    ws = analyzer(q)
    for w in ws:
        if w not in russ_vocab:
            return False
    return True

Run filters

In [5]:
def apply_filters(q, filters):
    new_q = q
    for f in filters:
        new_q = list(filter(f, new_q))
    return new_q

def print_q(qs):
    for q in qs:
        print(q)

In [140]:
all_filters = [
    filter_shorter_than_n,
    filter_starts_with_how,
    filter_starts_with_what,
    filter_stop_words,
    filter_stop_substr,
    filter_starts_with_stop_words
]

filtered_q = apply_filters(
    q=full_q,
    filters=all_filters
)

In [141]:
print(len(filtered_q))
cnt = 0
for q in filtered_q:
    if 'нужн' in q:
        cnt += 1
        print(q)
print(cnt)

353592
сколько нежных слов я не сказал сколько их ненужных обронил
сколько серий в сериале остров ненужных людей
куда отдать ненужные вещи
куда отдать ненужную одежду
где продать ненужные вещи
где снимался фильм остров ненужных людей
где снимали остров ненужных людей
сколько серий остров ненужных людей
где снимали фильм остров ненужных людей
сколько серий в фильме остров ненужных людей
что полезного и нужного вы могли бы предложить компании
будет ли продолжение сериала остров ненужных людей
сколько серий в острове ненужных людей
потому что потому что всех нужнее и дороже
куда сдать ненужную одежду
куда можно отдать ненужную одежду
программа которая удаляет ненужные файлы
куда деть ненужную одежду
куда сдать ненужные вещи
где снимался остров ненужных людей
куда деть ненужные книги
сколько серий в остров ненужных людей
где снимался сериал остров ненужных людей
на каком острове снимался фильм остров ненужных людей
где снимали сериал остров ненужных людей
куда деть ненужные вещи
куда можно

### Properties search

Search methods

In [8]:
def finder_substr(q, prop):
    if re.search(prop, q) is not None:
        return True
    return False

Run search

In [9]:
def apply_properties_finder(q, finder, prop):
    f = partial(finder, prop=prop)
    return list(filter(f, q))

In [10]:
def find_property(aliases):
    qs = []
    for alias in aliases:
        cur_qs = apply_properties_finder(filtered_q, finder_substr, alias)
        qs.extend(cur_qs)
    return qs

Load aliases

In [120]:
with open('properties_aliases.json', 'r') as inf:
    prop_aliases = json.load(inf)

Search

In [132]:
find_property(prop_aliases['P143'])

KeyError: 'P143'

In [130]:
prop_aliases['P1559']

['имя при рождении', 'на родном языке', 'оригинал имени', 'оригинальное имя']

In [83]:
print('\\\\')

\\
